In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
# %reload_ext autoreload
from imports import *
repertory_memoire="/Users/margauxvandererven/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/memoire/"

Dispersion is linear
Dispersion is linear


In [2]:
plt.figure(figsize=(20, 6))
# plt.plot(wavelen_visible, flux_visible, color='black', label='observé')

synthetic = syntspec("/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis.conv")
synthetic_norm = normalisation(wavelen_visible, flux_visible, 4600, 100)
# synthetic_norm2 = normalisation(synthetic_norm["z_wavelen"], synthetic_norm["flux_normalised"], 4600, 20)
plt.plot(synthetic_norm["z_wavelen"], synthetic_norm["flux_normalised"], color='black', label='observé')
# plt.plot(synthetic_norm["z_wavelen"], synthetic_norm["flux_normalised"], color='green', label='observé')
plt.plot(synthetic["wavelen"], synthetic["flux"], color='red', label='synthétique')
plt.hlines(1, 4500, 4700, color='lightgray', linestyle='--')
plt.xlim(4575, 4625)
plt.xlabel("Longueur d'onde (Å)")
plt.ylabel("Flux normalisé")
# plt.ylim(0.8, 200)
plt.legend()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis.conv'

In [3]:
# import numpy as np
# import scipy.interpolate as interp
# import matplotlib.pyplot as plt

def normalize_spectrum(wavelengths, flux, band_width=20):
    """
    Normalise le spectre en divisant par la médiane des 5 % plus grandes valeurs 
    dans une bande de 'band_width' Å autour de chaque point.
    """
    normalized_flux = np.zeros_like(flux)
    half_band = band_width / 2

    for i, wl in enumerate(wavelengths):
        mask = (wavelengths >= wl - half_band) & (wavelengths <= wl + half_band)
        if np.sum(mask) > 0:
            top_5_percent = np.percentile(flux[mask], 95)  # Seuil des 5% plus grands
            median_top_values = np.median(flux[mask][flux[mask] >= top_5_percent])  # Médiane des 5% plus grands
            normalized_flux[i] = flux[i] / median_top_values if median_top_values > 0 else flux[i]
        else:
            normalized_flux[i] = flux[i]

    return normalized_flux

def align_spectra(wl_obs, flux_obs, wl_syn, flux_syn, wl_ref, interp_kind="linear", align_band=20):
    """
    Aligne le spectre observé sur le spectre synthétique autour d'une longueur d'onde de référence `wl_ref`.
    1. Interpole le spectre observé sur la grille fine du synthétique.
    2. Normalise autour de `wl_ref` dans une bande de `align_band` Å.
    3. Ajuste verticalement le spectre observé pour l'aligner au synthétique.
    """
    wl_obs = np.array(wl_obs)
    flux_obs = np.array(flux_obs)
    wl_syn = np.array(wl_syn)
    flux_syn = np.array(flux_syn)
    obs_norm = normalisation(wl_obs,flux_obs, 4600, 100)

    interp_obs = interp.interp1d(obs_norm['z_wavelen'], obs_norm['flux_normalised'], kind=interp_kind, fill_value="extrapolate")
    flux_obs_interp_norm = interp_obs(wl_syn)

    half_band = align_band / 2
    mask = (wl_syn >= wl_ref - half_band) & (wl_syn <= wl_ref + half_band)

    if np.sum(mask) == 0:
        raise ValueError(f"Aucune donnée trouvée pour l'alignement dans [{wl_ref - half_band}, {wl_ref + half_band}] Å")

    diff_median = np.median(flux_obs_interp_norm[mask] - flux_syn[mask])  # Différence médiane
    aligned_flux = flux_obs_interp_norm - diff_median  # Correction

    return wl_syn, aligned_flux


synthetic = syntspec("/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis.conv")
synthetic_sansCH = syntspec("/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis_sans12CH.conv")
norm = normalisation(wavelen_visible, flux_visible, 4600, 100)

wl_obs = wavelen_visible
flux_obs = flux_visible
wl_syn = synthetic["wavelen"]  # Longueurs d'onde synthétiques
flux_syn = synthetic["flux"]  # Spectre synthétique

wl_syn_CH = synthetic_sansCH["wavelen"]  # Longueurs d'onde synthétiques
flux_syn_CH = synthetic_sansCH["flux"]  # Spectre synthétique
wl_syn_CH=np.array(wl_syn_CH)
flux_syn_CH=np.array(flux_syn_CH)
# Alignement du spectre observé sur la grille fine du synthétique
wl_aligned, flux_obs_aligned = align_spectra(wl_obs, flux_obs, wl_syn, flux_syn, wl_ref=4600)

# Affichage des résultats
plt.figure(figsize=(10, 5))
plt.hlines(1, 4500, 4700, color='gray', linestyle='--')

plt.plot(wl_syn, flux_obs_aligned, label="Observé normalisé et aligné", linewidth=1, color='black')

plt.plot(wl_syn, flux_syn, label="Synth avec 12CH", linewidth=1)
plt.plot(wl_syn_CH, flux_syn_CH, label="Synth sans 12CH", linewidth=1)
plt.plot(wl_syn, 1-np.abs(flux_syn-flux_syn_CH), label="Différence", linewidth=1)
# plt.plot(norm["z_wavelen"], norm["flux_normalised"], label="Observé normalisé", color='black', alpha=0.5)
plt.xlim(4500, 4700)
plt.ylim(0.0, 1.2)
# plt.scatter(wl_obs, flux_obs, label="Observé (original)", color="red", s=5, alpha=0.5)
plt.legend()
plt.xlabel("Longueur d'onde (Å)")
plt.ylabel("Flux")

plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis.conv'

In [2]:
minimum = find_peaks_element("/Users/margauxvandererven/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/memoire/BD-221742/synth_margaux/",
                   "4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_Feabu_7.2.conv",
                   "4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_sans12CH.conv")["minima"]
for i in minimum:
    if i < 5000:
        path_to_synth = "/Users/margauxvandererven/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/memoire/BD-221742/synth_margaux/"
        synth={"4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_Feabu_7.2.conv":"avec 12CH",
                    "4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_sans12CH.conv": "sans 12CH"}
        zoom_lines({"":[i]}, path_to_synth, synth,stardata,2,lines_BD22_vis,gamme="visible")

KeyboardInterrupt: 

In [17]:
# lines_BD22_vis = {"Na I" : get_ew_atom(1e-8, 4000, gamme="visible", particular_element="Na I")["wavelength"]}
data = get_ew_atom(1e-7, 4000, gamme="visible")["data"]
# print(lines_BD22_vis)
# with open("../data_lines/lines_BD22_vis.json", "w") as f:
#     json.dump(lines_BD22_vis, f)
lines_BD221742 = defaultdict(list)

# Trier les données par d[4] (élément) puis par d[0] (wavelength)
data_sorted = sorted(data, key=lambda x: (x[4], x[0]))

# Dictionnaire temporaire pour stocker la meilleure valeur de d[2] par élément et par plage de d[0]
best_values = {}

for wavelength, excitation_potential, ew, loggf, element in data_sorted:
    # Vérifier s'il existe déjà une valeur proche dans la liste
    if element in best_values:
        close_wavelengths = [w for w in best_values[element] if abs(w - wavelength) < 0.2]

        if close_wavelengths:
            # Vérifier si la nouvelle valeur de d[2] (ew) est plus grande que celle existante
            closest_wavelength = close_wavelengths[0]
            if ew > best_values[element][closest_wavelength]:  
                best_values[element][closest_wavelength] = ew  # Mise à jour avec le plus grand d[2]
        else:
            best_values[element][wavelength] = ew  # Ajouter un nouveau point si aucun proche n'existe
    else:
        best_values[element] = {wavelength: ew}  # Initialiser pour cet élément

# Convertir best_values en dictionnaire {d[4]: [d[0]]}
for element, wavelengths in best_values.items():
    lines_BD221742[element] = list(wavelengths.keys())
# pprint(lines_BD221742)

In [18]:
with open("../data_lines/lines_BD22_vis.json", "w") as fichier:
    json.dump(lines_BD221742, fichier, indent=4, ensure_ascii=False)

In [27]:
# get_ew_atom(1e-3, 4000, gamme="visible", particular_element="Fe I")["wavelength"]

set_complet = set(get_ew_atom(1e-9, 4000, gamme="visible", particular_element="Fe I")["wavelength"])
set_a_retirer = set(get_ew_atom(1e-8, 4000, gamme="visible", particular_element="Fe I")["wavelength"])

# Retirer les éléments
liste_resultante = list(set_complet - set_a_retirer)

# Trier la liste résultante si nécessaire
liste_resultante.sort()
print(liste_resultante)

[4000.098, 4001.175, 4002.02, 4002.634, 4002.792, 4003.164, 4003.178, 4003.617, 4004.594, 4005.136, 4005.602, 4007.24, 4008.867, 4009.072, 4009.73, 4010.784, 4012.164, 4013.845, 4013.889, 4014.779, 4016.613, 4016.793, 4017.21, 4018.494, 4020.047, 4020.149, 4023.952, 4025.951, 4027.098, 4028.098, 4028.803, 4032.239, 4032.993, 4036.43, 4037.129, 4037.168, 4037.746, 4037.817, 4038.368, 4039.194, 4040.828, 4043.251, 4045.367, 4045.614, 4045.669, 4046.292, 4046.457, 4047.318, 4048.732, 4048.886, 4049.771, 4050.541, 4050.732, 4050.927, 4051.26, 4052.245, 4052.275, 4054.413, 4055.082, 4055.474, 4056.812, 4057.869, 4058.464, 4060.15, 4060.374, 4062.663, 4069.074, 4070.205, 4075.512, 4076.36, 4077.655, 4078.657, 4078.817, 4080.062, 4080.081, 4080.654, 4081.132, 4081.517, 4084.208, 4085.064, 4086.436, 4086.825, 4087.139, 4087.917, 4090.107, 4090.753, 4093.266, 4093.834, 4095.966, 4096.692, 4096.905, 4097.236, 4100.971, 4102.763, 4102.865, 4103.267, 4104.76, 4106.175, 4106.912, 4107.284, 4107.513